In [2]:
import pandas as pd
import numpy as np
from PIL import Image

In [9]:
raw_trainData = pd.read_csv('train.csv')
raw_trainData

def read_data(mode="train", p=0.2, seed=None):
    """
    Read the training/testing files and return the input data and labels.

    Parameters:
    ----------
    mode (str): 'train' or 'test' to read the training or testing data.
    p (float): fraction of the data to read. Useful for debugging.
    seed (int): random seed to use for reproducibility.

    Returns:
    -------
    X (np.array): input data
    y (np.array): labels
    """
    if mode == "train":
        data = pd.read_csv('train.csv')
    elif mode == "test":
        data = pd.read_csv('test.csv')
    else:
        raise ValueError("mode must be 'train' or 'test'")

    if seed is not None:data = data.sample(frac=p)
    else: data = data.sample(frac=p, random_state=seed)

    X = data['im_name'].apply(lambda filename: np.asarray(Image.open(f"train_ims/{filename}"))).values
    y = data['label'].values
    X = np.stack(X)

    return X, y

In [10]:
trainData, trainLabels = read_data(seed=69)
trainData.shape, trainLabels.shape

((10000, 32, 32, 3), (10000,))